In [2]:
# import necessary libraries and modules
from PIL import Image
from pytesseract import pytesseract
import re
import os
import pandas as pd
import numpy as np

In [3]:
# set name of file and concatenate with the correct directory
file_name = 'receipt_6_family_mart.jpeg'
image_path = os.path.join(r'C:\Users\denti\python\OpticalCharacterRecognition\receipts', file_name)
# set path to tesseract.exe, which is fix and is necessary for pytesseract module to work
# tesseract has to be installed prior to using this code
path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [4]:
# convert image to text using tesseract
img = Image.open(image_path)
pytesseract.tesseract_cmd = path_to_tesseract
text = pytesseract.image_to_string(img).replace('X','x').replace('*','x').replace('%','x')
text

'Pringles Chee\nx 24,700 x |\n\n— Oishi PoppyPopJneBkr\n\n7, 800 i 2\n\n15, 600\n\n'

In [4]:
# check if a string is a float
def is_number(string):
    try:
        float(string) or int(string)
        return True
    except ValueError:
        pass

# extract the necessary information on items, amounts, and prices from text object and put it into a dataframe
def convert_text_to_dataframe_foodhall(text):
    splitted_text = text.split('\n')
    
    item = []
    amount_str = []
    price_total = []
    price_per_item_str = []
    text_lines = []
    
    # create lists with required information
    for line in splitted_text:
        if re.findall('\d{1,}x\d{1,}', line):

            start_index = [index for index, string in enumerate(line) if string.isdigit()]
            middle_index = [index for index, string in enumerate(line) if string == 'x']
            end_index = [index for index, string in enumerate(line) if string ==' ']

            text_lines.append(line)

            if start_index != [] and middle_index != [] and end_index != []:
                if start_index[0] > end_index[0]:
                    item.append(line[end_index[1]:].strip())
                    amount_str.append(line[start_index[0]:middle_index[0]])
                    price_per_item_str.append(line[middle_index[0]+1:end_index[1]].replace(' ', '').replace(',', ''))
                elif line[end_index[0]-1] == ',' or line[end_index[0]+1] == ',':
                    item.append(line[end_index[1]:].strip())
                    amount_str.append(line[start_index[0]:middle_index[0]])
                    price_per_item_str.append(line[middle_index[0]+1:end_index[1]].replace(' ', '').replace(',', ''))
                else:
                    item.append(line[end_index[0]:].strip())
                    amount_str.append(line[start_index[0]:middle_index[0]])
                    price_per_item_str.append(line[middle_index[0]+1:end_index[0]].replace(' ', '').replace(',', ''))

    # convert to correct format and calculate price per single item
    amount = []
    price_per_item = []

    for entry in amount_str:
        if is_number(entry.replace(',','.')):
            amount.append(float(entry.replace(',','.')))

    for entry in price_per_item_str:
        if is_number(entry.replace(',','.')):
            price_per_item.append(float(entry.replace(',','.')))
        
    i = 0
    for entry in price_per_item:
        price_total.append(round(float(price_per_item[i])*float(amount[i]), 2))
        i += 1       
        
    # create pandas data frame
    df = pd.DataFrame(list(zip(item, amount, price_total, price_per_item, text_lines)), 
                      columns = ['Item', 'Amount', 'Total Price', 'Price per Item', 'Source Text']
                     )      
    return df

# extract the necessary information on items, amounts, and prices from text object and put it into a dataframe
def convert_text_to_dataframe_family_mart(text):
    splitted_text = text.split('\n')

    text_lines = []
    i = 0
    for entry in splitted_text:
        if i % 2 == 1 and entry != '':
            text_lines.append(entry)
        i += 1

    item = []
    i = 0
    for entry in splitted_text:
        if i % 2 == 0 and entry != '':
            item.append(entry)
        i += 1

    price_per_item = []
    for entry in text_lines:
        if re.findall('[ \d]{1,}[x]{1,}[\d ]{1,}', entry):
            price_per_item.append(float(entry.split()[0].replace(',','')))

    amount = []
    for entry in text_lines:
        if re.findall('[ \d]{1,}[x]{1,}[\d ]{1,}', entry):
            amount.append(float(entry.split()[2].strip()))

    price_total = [amount[i]*price_per_item[i] for i in range(len(amount))]
    
    # create pandas data frame
    df = pd.DataFrame(list(zip(item, amount, price_total, price_per_item, text_lines)), 
                      columns = ['Item', 'Amount', 'Total Price', 'Price per Item', 'Source Text']
                     )
    
    return df

In [5]:
# extract the necessary information on items, amounts, and prices from text object and put it into a dataframe
def create_excel_from_dataframes(df):
    excel_name = 'Price_Comparison.xlsx'
    sheet_name = 'sheet_1'
    
    # read source excel file into first dataframe
    df_source = pd.DataFrame(pd.read_excel(excel_name, sheet_name=sheet_name))
    
    # combine first dataframe with new dataframe
    df_final = df_source.append(df, sort=False)
    
    # creater writer object to write in file
    writer = pd.ExcelWriter(excel_name)
    df_final.to_excel(writer, sheet_name=sheet_name, index=False)

    # set the column width in excel to autofit the content
    for column in df:
        column_length = max(df[column].astype(str).map(len).max(), len(column))
        column_index = df.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(column_index, column_index, column_length)

    writer.save()

In [6]:
# foodhall
splitted_text = text.split('\n')

item = []
amount_str = []
price_total = []
price_per_item_str = []
text_lines = []

# create lists with required information
for line in splitted_text:
    if re.findall('\d{1,}x\d{1,}', line):

        start_index = [index for index, string in enumerate(line) if string.isdigit()]
        middle_index = [index for index, string in enumerate(line) if string == 'x']
        end_index = [index for index, string in enumerate(line) if string ==' ']

        text_lines.append(line)
    
        if start_index != [] and middle_index != [] and end_index != []:
            if start_index[0] > end_index[0]:
                item.append(line[end_index[1]:].strip())
                amount_str.append(line[start_index[0]:middle_index[0]])
                price_per_item_str.append(line[middle_index[0]+1:end_index[1]].replace(' ', '').replace(',', ''))
            elif line[end_index[0]-1] == ',' or line[end_index[0]+1] == ',':
                item.append(line[end_index[1]:].strip())
                amount_str.append(line[start_index[0]:middle_index[0]])
                price_per_item_str.append(line[middle_index[0]+1:end_index[1]].replace(' ', '').replace(',', ''))
            else:
                item.append(line[end_index[0]:].strip())
                amount_str.append(line[start_index[0]:middle_index[0]])
                price_per_item_str.append(line[middle_index[0]+1:end_index[0]].replace(' ', '').replace(',', ''))

# convert to correct format and calculate price per single item
amount = []
price_per_item = []

for entry in amount_str:
    amount.append(float(entry.replace(',','.')))

for entry in price_per_item_str:
    price_per_item.append(float(entry.replace(',','.')))
    
i = 0

for entry in price_per_item:
    price_total.append(round(float(price_per_item[i])*float(amount[i]), 2))
    i += 1       

# create pandas data frame
df = pd.DataFrame(list(zip(item, amount, price_total, price_per_item, text_lines)), 
                  columns = ['Item', 'Amount', 'Total Price', 'Price per Item', 'Source Text']
                 )
df

,Item,Amount,Total Price,Price per Item,Source Text


In [10]:
# family mart
splitted_text = text.split('\n')

text_lines = []
i = 0
for entry in splitted_text:
    if i % 2 == 1 and entry != '':
        text_lines.append(entry)
    i += 1

item = []
i = 0
for entry in splitted_text:
    if i % 2 == 0 and entry != '':
        item.append(entry)
    i += 1
    
print(text_lines)
print(item)

price_per_item = []
for entry in text_lines:
    if re.findall('[ \d]{1,}[x]{1,}[\d ]{1,}', entry):
        price_per_item.append(float(entry.split()[0].replace(',','')))

amount = []
for entry in text_lines:
    if re.findall('[ \d]{1,}[x]{1,}[\d ]{1,}', entry):
        amount.append(float(entry.split()[2].strip()))

price_total = [amount[i]*price_per_item[i] for i in range(len(amount))]

# create pandas data frame
df = pd.DataFrame(list(zip(item, amount, price_total, price_per_item, text_lines)), 
                  columns = ['Item', 'Amount', 'Total Price', 'Price per Item', 'Source Text']
                 )

df

['x 24,700 x |', '— Oishi PoppyPopJneBkr', '7, 800 i 2', '15, 600']
['Pringles Chee']


In [10]:
# extract the necessary information on items, amounts, and prices from text object and put it into a dataframe
excel_name = 'Price_Comparison.xlsx'
sheet_name = 'sheet_1'

# read source excel file into first dataframe
df_source = pd.DataFrame(pd.read_excel(excel_name, sheet_name=sheet_name))

# combine first dataframe with new dataframe
df_final = df_source.append(df, sort=False)

# creater writer object to write in file
writer = pd.ExcelWriter(excel_name)
df_final.to_excel(writer, sheet_name=sheet_name, index=False)

# set the column width in excel to autofit the content
for column in df:
    column_length = max(df[column].astype(str).map(len).max(), len(column))
    column_index = df.columns.get_loc(column)
    writer.sheets[sheet_name].set_column(column_index, column_index, column_length)

writer.save()

In [9]:
#df = convert_text_to_dataframe(text)
#create_excel_from_dataframes(df)